# QA1. Changing associations

In [1]:
from koselleck import *
words=get_valid_words()

In [3]:
dfmodels = get_pathdf_models().query('period_len==5 & 1720<=period_start<1900')
dfmodels

,corpus,period_start,period_end,path,path_vocab,run,period,period_len
506,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_01,1720-1725,5
513,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_02,1720-1725,5
495,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_03,1720-1725,5
504,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_04,1720-1725,5
508,bpo,1720,1725,/home/ryan/github/koselleck/data/models/bpo/17...,/home/ryan/github/koselleck/data/models/bpo/17...,run_05,1720-1725,5
...,...,...,...,...,...,...,...,...
1656,bpo,1895,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_21,1895-1900,5
1665,bpo,1895,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_22,1895-1900,5
1663,bpo,1895,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_23,1895-1900,5
1648,bpo,1895,1900,/home/ryan/github/koselleck/data/models/bpo/18...,/home/ryan/github/koselleck/data/models/bpo/18...,run_24,1895-1900,5


In [60]:
def do_gen_localdist(pathdf,words=None,k=100,force=False):
    mpath=pathdf.iloc[0].path
    ofnfn=os.path.join(os.path.dirname(mpath), 'neighbs.pkl')
    res=pd.DataFrame([{'path_neighbs':ofnfn}])
    if not force and os.path.exists(ofnfn): return res
    
    if not os.path.exists(mpath): return pd.DataFrame()
    m=load_model(mpath)

    if not words: words=get_valid_words()
    words=set(words) & set(m.wv.key_to_index.keys())
    
    o=[]
    for w in words:
        neighb_l=list(m.wv.most_similar(w,topn=k))
        for i,(w2,c2) in enumerate(neighb_l):
            o+=[{'word':w, 'neighbor':w2, 'rank':i+1, 'csim':c2}]
    odf=pd.DataFrame(o)
    odf.to_pickle(ofnfn)
    return res

In [66]:
# pd.read_pickle('/home/ryan/github/koselleck/data/models/bpo/1720-1725/run_01/neighbs.pkl')

In [67]:
def gen_localdists(dfmodels,words=None,k=100,num_proc=1):
    if not words: words=get_valid_words()
    return pmap_groups(
        do_gen_localdist,
        dfmodels.groupby(['corpus','period','run']),
        num_proc=num_proc,
        kwargs=dict(
            k=k,
            words=words
        )
    )

In [ ]:
odf=gen_localdists(dfmodels, num_proc=7)

Mapping do_gen_localdist [x7]:   9%|▉         | 80/900 [04:55<1:11:10,  5.21s/it]

In [ ]:
odf

In [16]:
do_get_localdist((dfmodels.path.iloc[100], words, 50, True))

[('enchanting', 0.6885756850242615),
 ("warm'd", 0.668242335319519),
 ('perceptions', 0.6664187908172607),
 ('ambient', 0.6618196368217468),
 ('canicular', 0.6591447591781616),
 ('cums', 0.6514627933502197),
 ('radiance', 0.6481860876083374),
 ('withes', 0.6460468173027039),
 ('admiring', 0.6440391540527344),
 ('incessantly', 0.6419475078582764),
 ("angel's", 0.6405750513076782),
 ('pervade', 0.6385079026222229),
 ('warms', 0.6374044418334961),
 ('sensa', 0.6373929977416992),
 ('surpasses', 0.6366644501686096),
 ('intrepidity', 0.6358006596565247),
 ('zephyrs', 0.6350411772727966),
 ('lustre', 0.6346498131752014),
 ('renews', 0.6342523694038391),
 ('distraction', 0.6334822177886963),
 ("gath'ring", 0.6326651573181152),
 ('excites', 0.6298933625221252),
 ('divinely', 0.6287623047828674),
 ('poisons', 0.6273611783981323),
 ('sweeps', 0.6269160509109497),
 ('scorns', 0.6261158585548401),
 ('droop', 0.6251063346862793),
 ('swoln', 0.6241396069526672),
 ('glowing', 0.6222972273826599),
 ('f